# deeplmt
> Library to use DeepL machine translation services

In [ ]:
#| default_exp deeplmt

In [ ]:
#| hide
import os

In [ ]:
#| hide
running_in_colab = 'google.colab' in str(get_ipython())
if running_in_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    homedir = "/content/drive/MyDrive"
else:
    homedir = os.getenv('HOME')

In [ ]:
#| hide
if running_in_colab:
    github_test_folder = homedir+"/github/polyglottech/mteval"
    %cd {github_test_folder}
    !pip3 install nbdev
    !pip3 install -e '.[dev]'

In [ ]:
#| hide
from dotenv import load_dotenv

In [ ]:
#| hide
if running_in_colab:
    # Colab doesn't have a mechanism to set environment variables other than python-dotenv
    env_file = homedir+'/secrets/.env'
    %load_ext dotenv
    %dotenv {env_file}

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import deepl
from pathlib import Path
from langcodes import *

class deepltranslate:
    """Class to get translations from the DeepL API"""
    def __init__(self):
        """Constructor of deepltranslate class"""
        self._apikey = os.getenv('DEEPL_API_KEY')
        self._translator = deepl.Translator(self._apikey)
        # Caches for supported source and target language ids
        self._source_languages = []
        self._target_languages = []

    def check_langpair(self,sourcelang, targetlang):
        """Function to check if the language pair is supported"""
        if tag_is_valid(sourcelang) and tag_is_valid(targetlang):
            source_langid_upper = sourcelang.upper()
            target_langid_upper = targetlang.upper()
            supported_source_languages = self._source_languages
            if not supported_source_languages:
                supported_source_languages = [lang.code for lang in self._translator.get_source_languages()]
                self._source_languages = supported_source_languages
            supported_target_languages = self._target_languages
            if not supported_target_languages:
                supported_target_languages = [lang.code for lang in self._translator.get_target_languages()]
                self._target_languages = supported_target_languages

            if source_langid_upper in supported_source_languages:
                if target_langid_upper in supported_target_languages:
                    return True
        
        return False

    def translate_text(self,sourcelang, targetlang, text):
        """Function to translate text into the target language"""
        translated_text = ""
        if self.check_langpair(sourcelang,targetlang):
            source_langid_upper = sourcelang.upper()
            target_langid_upper = targetlang.upper()
            result = self._translator.translate_text(text,source_lang=source_langid_upper,target_lang=target_langid_upper,split_sentences='off')
            translated_text = result.text

        return translated_text


In [ ]:
#| hide
deeplmt = deepltranslate()
translation = deeplmt.translate_text("en","it","This is a cool car!")
assert translation == "Questa è una macchina fantastica!"

assert True == deeplmt.check_langpair("en","it")
assert False == deeplmt.check_langpair("en","gu")

In [ ]:
show_doc(deepltranslate.translate_text)

In [ ]:
show_doc(deepltranslate.check_langpair)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()